In [1]:
from mxnet import nd
from mxnet import gluon
from mxnet import image
import matplotlib.pyplot as plt
% matplotlib inline


d:\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
d:\python35\lib\site-packages\mxnet\optimizer.py:136: UserWarning: WARNING: New optimizer mxnet.optimizer.NAG is overriding existing optimizer mxnet.optimizer.NAG
  Optimizer.opt_registry[name].__name__))


In [2]:
help(image.CreateAugmenter)

Help on function CreateAugmenter in module mxnet.image.image:

CreateAugmenter(data_shape, resize=0, rand_crop=False, rand_resize=False, rand_mirror=False, mean=None, std=None, brightness=0, contrast=0, saturation=0, hue=0, pca_noise=0, rand_gray=0, inter_method=2)
    Creates an augmenter list.
    
    Parameters
    ----------
    data_shape : tuple of int
        Shape for output data
    resize : int
        Resize shorter edge if larger than 0 at the begining
    rand_crop : bool
        Whether to enable random cropping other than center crop
    rand_resize : bool
        Whether to enable random sized cropping, require rand_crop to be enabled
    rand_gray : float
        [0, 1], probability to convert to grayscale for all channels, the number
        of channels will not be reduced to 1
    rand_mirror : bool
        Whether to apply horizontal flip to image with probability 0.5
    mean : np.ndarray or None
        Mean pixel values for [r, g, b]
    std : np.ndarray or None

In [3]:
def image_augmentaion(image_data, n =4):
    # An example of creating multiple augmenters
    image_data = image_data.astype('float32')
    
#     augs = image.CreateAugmenter(data_shape=image_data.shape, rand_mirror=True)

    augs = [
#         image.ResizeAug(250), # 将短边resize至250
        image.HorizontalFlipAug(.5), # 0.5概率的水平翻转变换
#         image.HueJitterAug(.6), # -0.6~0.6的随机色调
        image.BrightnessJitterAug(.25), # -0.5~0.5的随机亮度
#         image.RandomCropAug([230,230]) # 随机裁剪成（230,230）
#         image.ContrastJitterAug(.25), #调整对比度
#         image.RandomSizedCropAug((200,200), .1, (.5,2)) #随机裁剪，要求保留至少0.1的区域，随机长宽比在.5和2之间
    ]
    # dump the details
    print(augs,len(augs),'\n end')
    img = [aug(image_data) for aug in augs]    

    img_aug = nd.stack(*img).clip(0,255)/255
    return img_aug

In [4]:
def apply(img, aug, n=2):
    # 转成float，一是因为aug需要float类型数据来方便做变化。
    # 二是这里会有一次copy操作，因为有些aug直接通过改写输入
    #（而不是新建输出）获取性能的提升
    X = [aug(img.astype('float32')) for _ in range(n*n)]
    # 有些aug不保证输入是合法值，所以做一次clip
    # 显示浮点图片时imshow要求输入在[0,1]之间
    Y = nd.stack(*X).clip(0,255)/255
    return Y

In [5]:
def apply_aug_list(img, augs):
    for f in augs:
        img = f(img)
    return img

def transform(data, augs):
    # data: sample x height x width x channel
    # label: sample
    data = data.astype('float32')
    if augs is not None:
        # apply to each sample one-by-one and then stack
        data = nd.stack(*[apply_aug_list(d, augs) for d in data])
#         data = nd.transpose(data, (0,3,1,2))
    return data


In [11]:
img = image.imread('params.png')
img = image.imresize(img,640,490)
import numpy as np
img_data = []
for _ in range(4):
    img_data.append(img.asnumpy())

img_data = nd.array(img_data)
print(img_data[1])
augs = [
#         image.ResizeAug(250), # 将短边resize至250
        image.HorizontalFlipAug(.5), # 0.5概率的水平翻转变换
#         image.HueJitterAug(.6), # -0.6~0.6的随机色调
        image.BrightnessJitterAug(.25), # -0.5~0.5的随机亮度
#         image.RandomCropAug([230,230]) # 随机裁剪成（230,230）
#         image.ContrastJitterAug(.25), #调整对比度
#         image.RandomSizedCropAug((200,200), .1, (.5,2)) #随机裁剪，要求保留至少0.1的区域，随机长宽比在.5和2之间
    ]
# img_aug = transform(img_data, augs)
# # img_aug = image_augmentaion(img,4)
# print('\n --------------\n', len(img_aug))
# _ , figs = plt.subplots(1,4,figsize = (8,8))
# for i in range(len(img_aug)): 
#     figs[i].imshow(img_aug[i].asnumpy())


[[[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 ...

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]]
<NDArray 490x640x3 @cpu(0)>
